In [2]:
import import_ipynb
from lib import *

class Ewo:
    #All methods of Ewo should be used in order
    
    #Initialization
    def __init__(self, target_market_ticker, start_date, end_date):
        self.target_market_ticker = target_market_ticker
        self.start_date = start_date
        self.end_date = end_date
        self.market = pdr.get_data_yahoo(target_market_ticker, start_date, end_date)
        
    #Process data and return market & market_buy_init dataframes    
    def data_process(self, fast_sma, slow_sma, longer_sma, n_day_EWO_slope, n_day_closing_price, use_filter4):
        market = self.market.copy()
        self.fast_sma = fast_sma
        self.slow_sma = slow_sma
        self.longer_sma = longer_sma
        self.n_day_EWO_slope = n_day_EWO_slope
        self.n_day_closing_price = n_day_closing_price
        self.var1 = f'{fast_sma}-day'
        self.var2 = f'{slow_sma}-day'
        self.var3 = f'{longer_sma}-day'
        self.use_filter4 = use_filter4
        
        day = np.arange(1, len(market) + 1)
        market['Day'] = day 
        market[self.var1] = market['Close'].rolling(fast_sma).mean()   #Column for SMA_fast
        market[self.var2] = market['Close'].rolling(slow_sma).mean()   #Column for SMA_slow
        market[self.var3] = market['Close'].rolling(longer_sma).mean() #Column for SMA_longer-term
        market['EWO'] = market[self.var1] - market[self.var2]          #Column for EWO
        market = market[['Day', 'Open', 'Close', self.var1, self.var2, self.var3, 'EWO']]
        market = market.reset_index()
        market = market.set_index('Day')

        #Check whether EWO of each day is over a certain positive value, which is 1% of SMA_fast
        #(value 1 = Yes, value 0 = No)
        market['Filter1'] = np.where(market['EWO'] > market[self.var1]/100, 1, 0)

        #Check whether the slope of the line of best fit corresponding to N most recent EWO values is positive
        #(value 1 = Yes, value 0 = No)
        market['Filter2'] = np.where(market['EWO'].rolling(n_day_EWO_slope).apply(lobf) > 0, 1, 0)

        #Check whether the current price is in uptrend (i.e. Closing Price > SMA_longer-term)
        #(value 1 = Yes, value 0 = No)
        market['Filter3'] = np.where(market['Close'] > market[self.var3], 1, 0)
        
        if use_filter4:
            #Check whether there's a notable drop in the market
            #(value 1 = Yes, value 0 = No)
            market['Filter4'] = np.where(market['EWO'] < market[self.var1]*(-10/100), 1, 0)
        
        market.dropna(inplace=True)

        #Indicate where the N-day EWO's slope is negative AND where the price is in downtrend
        #(value -1 = Yes, value 0 = No)
        market['Exit1'] = np.where(market['EWO'].rolling(n_day_EWO_slope).apply(lobf) < 0, -1, 0)
        market['Exit2'] = np.where(market['Close'].rolling(n_day_closing_price).apply(lobf) < 0, -1, 0)

        #market_buy_init dataframe refers to the initial buy signals, where all the filters are satistfied
        condition1 = (market['Filter1'] == 1)
        condition2 = (market['Filter2'] == 1)
        condition3 = (market['Filter3'] == 1)
        if use_filter4:
            condition4 = (market['Filter4'] == 1)
            market_buy_init = market.loc[condition1&condition2&condition3|condition4]
        else:
            market_buy_init = market.loc[condition1&condition2&condition3]

        return market, market_buy_init
        
            
    #Backtesting of trading strategy on the target_market_ticker from start_date to end_date
    #Return market_buy and market_sell dataframes
    def backtest(self, budget, market, market_buy_init):
        buy_list = []   #List storing all the buy signals
        sell_list = []  #List storing all the sell signals
        sold_day = 0
        bought_amount = 0
        trade_num = 0
        if self.use_filter4:
            #This varaible tells whether the trade started from condition 4 is ongoing
            #(value 1 = Yes, value 0 = No)
            cond4 = 0

        #Iterate the elements in market_buy_init, which contains initial buy signals
        for i in range(len(list(market_buy_init.index))-1): 
            #If there are consecutive buy signals w/o a sell signal, continue the first buy signal
            if self.use_filter4:
                if (sold_day > list(market_buy_init.index)[i]+1) or \
                (market_buy_init.loc[list(market_buy_init.index)[i], 'Filter4'] == 1) and (cond4 == 1): 
                    continue
            elif sold_day > list(market_buy_init.index)[i]+1: 
                continue
  
            if bought_amount == 0: #Enter a trade only when there is no current position
                #Buy the security at the next morning's open price
                bought_day = list(market_buy_init.index)[i]+1
                buy_list.append(bought_day)
                bought_price = market.loc[bought_day, 'Open']
                bought_amount = budget // bought_price #Buy as many security as the budget allows at that price
                budget -= bought_amount * bought_price #Remaining budget after entering a long position
                #print(int(bought_amount), "amount of security is bought at",
                     #bought_price, "on", market.loc[bought_day, "Date"], "(Day", bought_day, ")")
                #print(budget, "is the remaining budget")
                
                if self.use_filter4:
                    if market.loc[bought_day-1, 'Filter4'] == 1:
                        cond4 = 1 #The trade started from condition 4 begins
                        continue
                        
            if self.use_filter4:
                if market_buy_init.loc[list(market_buy_init.index)[i], 'Filter4'] == 0:
                    bought_day = list(market_buy_init.index)[i]+1

            #Beginning from the day of long position until the end_date,
            #check whether both exit signals are satisfied simultaneously or 
            #whether it is the last day of trading
            for j in range(bought_day, market.iloc[[-1]].index[0]):
                if (market.loc[j, 'Exit1'] == -1) and (market.loc[j, 'Exit2'] == -1)\
                or (j == market.iloc[[-1]].index[0]-1):
                    #Sell the security at the next morning's open price
                    sold_day = j+1
                    sold_price = market.loc[sold_day, 'Open']
                    sell_list.append(sold_day)
                    budget += bought_amount * sold_price
                    #print(int(bought_amount), "amount of security is sold at",
                          #sold_price, 'on', market.loc[sold_day, "Date"], "(Day", sold_day, ")")
                    #print("After closing the position, current budget is", budget)
                    #print("\n")
                    #Reset the parameters
                    bought_amount = 0
                    trade_num += 1
                    if self.use_filter4:
                        cond4 = 0
                    break

        if bought_amount != 0: #In case the position is not cleared after the end_date
            sold_day = market.iloc[[-1]].index[0]
            sold_price = market.loc[sold_day, 'Open']
            sell_list.append(sold_day)
            budget += bought_amount * sold_price
            #print(int(bought_amount), 'amount of security is sold at', \
                #sold_price, 'on', market.loc[sold_day, "Date"], "(Day", sold_day, ")")
            #print("After closing the position, current budget is", budget)
            #print("\n")
            #Reset the parameter
            bought_amount = 0
            trade_num += 1
            if self.use_filter4:
                cond4 = 0
            
        print("The final budget of the trading strategy is", round(budget), "USD")
        print("Total number of trades is", trade_num)

        #All buy signals after the trade
        market_buy = market.copy()
        market_buy = market_buy.loc[buy_list] 

        #All sell signals after the trade
        market_sell = market.copy()
        market_sell = market_sell.loc[sell_list]
        
        return market_buy, market_sell
    
    def visualize(self, market, market_buy, market_sell):
        #Visualize the price trend, EWO value, buy signals, and sell signals throughout the trading period
        plt.rcParams['figure.figsize'] = 12, 6

        plt.subplot(211) #Upper plot
        plt.grid(True, alpha = .3)
        plt.plot(market['Date'], market['Close'], label = self.target_market_ticker) #market
        plt.plot(market['Date'], market[self.var1], label = self.var1) #SMA_fast
        plt.plot(market['Date'], market[self.var2], label = self.var2) #SMA_slow
        plt.plot(market['Date'], market[self.var3], label = self.var3) #SMA_longer-term
        plt.plot(market_buy['Date'], market_buy['Close'], '^',
                 color = 'brown', label='Buy', markersize = 9) #Buy signals
        plt.plot(market_sell['Date'], market_sell['Open'], '.',
                 color = 'k', label='Sell', markersize = 9)    #Sell signals
        plt.legend(loc='upper left')

        plt.subplot(212) #Lower plot for EWO 
        plt.grid(True, alpha = .3)
        plt.plot(market['Date'], market['EWO'], color='blue', label = 'EWO') #EWO
        plt.plot(market_buy['Date'], market_buy['EWO'], '^',
                 color = 'brown', label='Buy', markersize = 9) #Buy signals
        plt.plot(market_sell['Date'], market_sell['EWO'], '.',
                 color = 'k', label='Sell', markersize = 9)    #Sell signals
        plt.axhline(0, color='lightgray', linestyle='--', linewidth=2) #x-axis
        plt.legend(loc='lower left')
        plt.show()